<a href="https://colab.research.google.com/github/official-okello/DS_bootcamp_with_gomycode/blob/master/Retail_Sales_Analysis_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

It is fascinating to analyze a retail sales data associated with alcohol and liquor. It provides insights into both seasonal patterns and supplier behaviors that might otherwise go unnoticed. In this colab, I explore the 'Retail Sales Data set of Alcohol and Liquor' dataset, addressing potential issues such as missing values, and developing a predictive model for retail sales based on warehouse sales and retail transfers.

Colum Descriptions:

- **YEAR, MONTH**: Time reference of the transaction
- **SUPPLIER**: Distributor or manufacturer providing the item
- **ITEM CODE**, ITEM DESCRIPTION: Unique identifier and name of the product
- **ITEM TYPE**: Product category (e.g., Wine, Liquor, Beer)
- **RETAIL SALES**: Units sold in retail
- **RETAIL TRANSFERS**: Units transferred within retail locations
- **WAREHOUSE SALES**: Units moved or sold through the warehouse

# Loading in imports, general info gathering

In [67]:
#@title Imports

# Suppress warnings for a cleaner notebook output
import warnings
warnings.filterwarnings('ignore')

# Import necessary libraries
import pandas as pd
import numpy as np

# Install modules only if they are not already installed
try:
    import ydata_profiling
except ImportError:
    !pip install ydata-profiling

try:
    import scipy
except ImportError:
    !pip install scipy

import matplotlib
matplotlib.use('Agg')  # Use Agg backend for matplotlib
import matplotlib.pyplot as plt
plt.switch_backend('Agg') # Ensures matplotlib uses the Agg backend explicitly
%matplotlib inline

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score

print('\n All libraries imported successfully!')


 All libraries imported successfully!


In [68]:
#@title Load Dataset
df = pd.read_csv('/content/Warehouse_and_Retail_Sales.csv')

In [69]:
#@title First 5 rows
df.head()

,YEAR,MONTH,SUPPLIER,ITEM CODE,ITEM DESCRIPTION,ITEM TYPE,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES
0,2020,1,REPUBLIC NATIONAL DISTRIBUTING CO,100009,BOOTLEG RED - 750ML,WINE,0.00,0.0,2.0
1,2020,1,PWSWN INC,100024,MOMENT DE PLAISIR - 750ML,WINE,0.00,1.0,4.0
2,2020,1,RELIABLE CHURCHILL LLLP,1001,S SMITH ORGANIC PEAR CIDER - 18.7OZ,BEER,0.00,0.0,1.0
3,2020,1,LANTERNA DISTRIBUTORS INC,100145,SCHLINK HAUS KABINETT - 750ML,WINE,0.00,0.0,1.0
4,2020,1,DIONYSOS IMPORTS INC,100293,SANTORINI GAVALA WHITE - 750ML,WINE,0.82,0.0,0.0


In [70]:
#@title Last 5 rows
df.tail()

,YEAR,MONTH,SUPPLIER,ITEM CODE,ITEM DESCRIPTION,ITEM TYPE,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES
307640,2020,9,DOPS INC,97896,ST PETERS ORGANIC ENG ALE NR 12/CS - 16.9OZ,BEER,0.00,0.0,1.00
307641,2020,9,ANHEUSER BUSCH INC,97918,STELLA ARTOIS 2/12 NR - 11.2OZ,BEER,372.45,315.0,3586.88
307642,2020,9,HEINEKEN USA,97942,TECATE 4/6 LNNR - 12OZ,BEER,7.79,0.0,4.00
307643,2020,9,RELIABLE CHURCHILL LLLP,97950,S SMITH WINTER WELCOME NR 12/CS - 18.7OZ,BEER,0.00,0.0,2.00
307644,2020,9,RELIABLE CHURCHILL LLLP,97969,S SMITH WINTER WELCOME 4/6NR - 12OZ,BEER,0.00,0.0,1.00


In [71]:
#@title General Info on the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307645 entries, 0 to 307644
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   YEAR              307645 non-null  int64  
 1   MONTH             307645 non-null  int64  
 2   SUPPLIER          307478 non-null  object 
 3   ITEM CODE         307645 non-null  object 
 4   ITEM DESCRIPTION  307645 non-null  object 
 5   ITEM TYPE         307644 non-null  object 
 6   RETAIL SALES      307642 non-null  float64
 7   RETAIL TRANSFERS  307645 non-null  float64
 8   WAREHOUSE SALES   307645 non-null  float64
dtypes: float64(3), int64(2), object(4)
memory usage: 21.1+ MB


We see that there are 307645 total entries divided into 9 columns. We have datatypes int, float and object(that will be later converted to numeric for machine learning).

Also a couple of columns don't have full records indicating missing value which will be handled in the data cleaning phase.

In [72]:
#@title Shape of dataset
# Rechecking again to ascertain shape of the dataset
shape = df.shape
print(f'The dataset has {shape[0]} rows and {shape[1]} columns.')

The dataset has 307645 rows and 9 columns.


In [73]:
#@title General Statistical Description of the dataset
df.describe(include='all') # Include='all' to display also columns with object datatype

,YEAR,MONTH,SUPPLIER,ITEM CODE,ITEM DESCRIPTION,ITEM TYPE,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES
count,307645.000000,307645.000000,307478,307645,307645,307644,307642.000000,307645.000000,307645.000000
unique,NaN,NaN,396,34056,34822,8,NaN,NaN,NaN
top,NaN,NaN,REPUBLIC NATIONAL DISTRIBUTING CO,414050,BURGANS ALBARINO - 750ML,WINE,NaN,NaN,NaN
freq,NaN,NaN,20995,24,44,187640,NaN,NaN,NaN
mean,2018.438525,6.423862,NaN,NaN,NaN,NaN,7.024071,6.936465,25.294597
std,1.083061,3.461812,NaN,NaN,NaN,NaN,30.986238,30.237195,249.916798
min,2017.000000,1.000000,NaN,NaN,NaN,NaN,-6.490000,-38.490000,-7800.000000
25%,2017.000000,3.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000
50%,2019.000000,7.000000,NaN,NaN,NaN,NaN,0.320000,0.000000,1.000000
75%,2019.000000,9.000000,NaN,NaN,NaN,NaN,3.267500,3.000000,5.000000


Insights:
1. There are 391 different suppliers, out of which REPUBLIC NATIONAL DISTRIBUTING CO is the most frequent with 18027 records.
2. There are 8 different types of liquor and alchohol out of which WINE is the most frequent with 157980 records.
3. The dataset has records from years 2017 to 2020, all year(months 1 to 12).
4. The average retail sales is 6.997
5. The average retail transfer is 6.921
6. The average warehouse sale is 24.913

# Data Cleaning and Pre-processing

In [74]:
#@title Merge Year and Month columns into one datetime column
df['DATE'] = pd.to_datetime(df['YEAR'].astype(str) + '-' + df['MONTH'].astype(str) + '-01', format='%Y-%m-%d', errors='coerce')

# Drop Month and Year columns
df.drop(['YEAR', 'MONTH'], axis=1, inplace=True)

# Dataset after merging Year and Month columns into one datetime column
print('Dataset after merging Year and Month culumns: \n')
df.head()

Dataset after merging Year and Month culumns: 



,SUPPLIER,ITEM CODE,ITEM DESCRIPTION,ITEM TYPE,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES,DATE
0,REPUBLIC NATIONAL DISTRIBUTING CO,100009,BOOTLEG RED - 750ML,WINE,0.00,0.0,2.0,2020-01-01
1,PWSWN INC,100024,MOMENT DE PLAISIR - 750ML,WINE,0.00,1.0,4.0,2020-01-01
2,RELIABLE CHURCHILL LLLP,1001,S SMITH ORGANIC PEAR CIDER - 18.7OZ,BEER,0.00,0.0,1.0,2020-01-01
3,LANTERNA DISTRIBUTORS INC,100145,SCHLINK HAUS KABINETT - 750ML,WINE,0.00,0.0,1.0,2020-01-01
4,DIONYSOS IMPORTS INC,100293,SANTORINI GAVALA WHITE - 750ML,WINE,0.82,0.0,0.0,2020-01-01


In [75]:
#@title Missing Values
missing_values = df.isnull().sum() # Sum missing values per column
if missing_values.any():
    print(f'Missing values found in the dataset were {missing_values.sum()} in total, as per the columnwise divisions below:')
    print(missing_values)

    print(f'\n Deleting rows with missing values... \n') # Delete rows with missing values
    df = df.dropna()

    print(f'After deletion the dataset now has {df.shape[0]} rows')
else:
    print('No missing values found in the dataset.')

Missing values found in the dataset were 171 in total, as per the columnwise divisions below:
SUPPLIER            167
ITEM CODE             0
ITEM DESCRIPTION      0
ITEM TYPE             1
RETAIL SALES          3
RETAIL TRANSFERS      0
WAREHOUSE SALES       0
DATE                  0
dtype: int64

 Deleting rows with missing values... 

After deletion the dataset now has 307477 rows


In [76]:
#@title Duplicates
duplicates = df.duplicated().sum()
if duplicates > 0:
  df.drop_duplicates(inplace=True)
  print(f'Removed {duplicates} duplicate rows.')
  print(f'After deletion of duplicate rows,the dataset now has {df.shape[0]} rows')
else:
  print('No duplicates found in the dataset.')

No duplicates found in the dataset.


In [77]:
#@title Outliers using z-score
# Using zscore from scipy.stats
z_scores = np.abs(scipy.stats.zscore(df.select_dtypes(include=[np.number])))
outliers = np.where(z_scores > 3) # 3 as threshold to identify outliers

if len(outliers[0]) > 0:
    print(f'\n {len(outliers[0])} outliers were found in the dataset.')
    # remove outliers
    df = df[(z_scores < 3).all(axis=1)]
    print(f'\n {len(outliers[0])} outliers were removed from the dataset.')
    print(f'After removal of outliers,the dataset now has {df.shape[0]} rows')
else:
    print('\n No outliers were found in the dataset.')


 8762 outliers were found in the dataset.

 8762 outliers were removed from the dataset.
After removal of outliers,the dataset now has 302489 rows


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 302489 entries, 0 to 307644
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   SUPPLIER          302489 non-null  object        
 1   ITEM CODE         302489 non-null  object        
 2   ITEM DESCRIPTION  302489 non-null  object        
 3   ITEM TYPE         302489 non-null  object        
 4   RETAIL SALES      302489 non-null  float64       
 5   RETAIL TRANSFERS  302489 non-null  float64       
 6   WAREHOUSE SALES   302489 non-null  float64       
 7   DATE              302489 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 20.8+ MB


In [79]:
#@title Conversation of Categorical features to Numerical Features
# Will convert ITEM TYPE to numerical
df = pd.get_dummies(df, columns=['ITEM TYPE'])

# First 5 record after encoding
print('First 5 record after encoding:')
df.head()

First 5 record after encoding:


,SUPPLIER,ITEM CODE,ITEM DESCRIPTION,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES,DATE,ITEM TYPE_BEER,ITEM TYPE_DUNNAGE,ITEM TYPE_KEGS,ITEM TYPE_LIQUOR,ITEM TYPE_NON-ALCOHOL,ITEM TYPE_REF,ITEM TYPE_STR_SUPPLIES,ITEM TYPE_WINE
0,REPUBLIC NATIONAL DISTRIBUTING CO,100009,BOOTLEG RED - 750ML,0.00,0.0,2.0,2020-01-01,False,False,False,False,False,False,False,True
1,PWSWN INC,100024,MOMENT DE PLAISIR - 750ML,0.00,1.0,4.0,2020-01-01,False,False,False,False,False,False,False,True
2,RELIABLE CHURCHILL LLLP,1001,S SMITH ORGANIC PEAR CIDER - 18.7OZ,0.00,0.0,1.0,2020-01-01,True,False,False,False,False,False,False,False
3,LANTERNA DISTRIBUTORS INC,100145,SCHLINK HAUS KABINETT - 750ML,0.00,0.0,1.0,2020-01-01,False,False,False,False,False,False,False,True
4,DIONYSOS IMPORTS INC,100293,SANTORINI GAVALA WHITE - 750ML,0.82,0.0,0.0,2020-01-01,False,False,False,False,False,False,False,True


# Building a model to predict retail sales

In [80]:
#@title Split dataset into training and test sets
X = df.drop(['SUPPLIER', 'ITEM CODE', 'ITEM DESCRIPTION', 'RETAIL SALES', 'DATE'], axis=1)
y = df['RETAIL SALES']

print('Features: \n')
print(X.head())

print('\n Target: \n')
print(y.head())

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Features: 

   RETAIL TRANSFERS  WAREHOUSE SALES  ITEM TYPE_BEER  ITEM TYPE_DUNNAGE  \
0               0.0              2.0           False              False   
1               1.0              4.0           False              False   
2               0.0              1.0            True              False   
3               0.0              1.0           False              False   
4               0.0              0.0           False              False   

   ITEM TYPE_KEGS  ITEM TYPE_LIQUOR  ITEM TYPE_NON-ALCOHOL  ITEM TYPE_REF  \
0           False             False                  False          False   
1           False             False                  False          False   
2           False             False                  False          False   
3           False             False                  False          False   
4           False             False                  False          False   

   ITEM TYPE_STR_SUPPLIES  ITEM TYPE_WINE  
0                   False     

In [81]:
#@title Fit a Random Forest Regressor
rfr_model = RandomForestRegressor(n_estimators=100, random_state=42)
rfr_model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [82]:
#@title Evaluate Model Performance
rfr_y_pred = rfr_model.predict(X_test)
rfr_mse = mean_squared_error(y_test, rfr_y_pred)
rfr_r2 = r2_score(y_test, rfr_y_pred)

print(f'Random Forest Regressor - Mean Squared Error: {rfr_mse}')
print(f'Random Forest Regressor - R-squared: {rfr_r2}')

Random Forest Regressor - Mean Squared Error: 12.827819020009562
Random Forest Regressor - R-squared: 0.9033598528401412


In [66]:
#@title Crossvalidation to optimize model
rfr_cv_scores = cross_val_score(rfr_model, X, y, cv=5, scoring='neg_mean_squared_error')
rfr_cv_mse = -rfr_cv_scores.mean()
rfr_cv_r2 = cross_val_score(rfr_model, X, y, cv=5, scoring='r2').mean()

print(f'Random Forest Regressor - Cross-validated Mean Squared Error: {rfr_cv_mse}')
print(f'Random Forest Regressor - Cross-validated R-squared: {rfr_cv_r2}')

Random Forest Regressor - Cross-validated Mean Squared Error: 12.755652259526332
Random Forest Regressor - Cross-validated R-squared: 0.8998079733108334
